# Lab | Working with APIs
**Instructions**

Following the class example, create a function that returns the price, names of origin and arrival airports and the name of the company. Do it for all the flights between two dates that cost the same.

In [1]:
from config import *
import requests
import pandas as pd
import json

In [2]:
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format()) # Format makes the output ans str variable.

get_dates('1992-01-01','1992-01-20')

0     1992-01-01
1     1992-01-02
2     1992-01-03
3     1992-01-04
4     1992-01-05
5     1992-01-06
6     1992-01-07
7     1992-01-08
8     1992-01-09
9     1992-01-10
10    1992-01-11
11    1992-01-12
12    1992-01-13
13    1992-01-14
14    1992-01-15
15    1992-01-16
16    1992-01-17
17    1992-01-18
18    1992-01-19
19    1992-01-20
dtype: object

In [3]:
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"

    params = {"query": city_name}

    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
                      'x-rapidapi-key': apiKey}

    response = requests.get(url, headers = headers, params = params)

    return response.json()["Places"][0]["PlaceId"]

In [4]:
city_code('Paris')

'PARI-sky'

In [5]:
def flight_prices(departure, arrival, date):

    departure_code = city_code(departure)
    arrival_code = city_code(arrival)
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date}"

    params = {"inboundpartialdate":{date}}

    #headers = {
    #'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    #'x-rapidapi-key': "<introduce your RapidAPI key here>"}

    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': apiKey}

    response = requests.get(url, headers=headers, params=params)

    return response.json()

flight_prices('Paris', 'London','2021-12-10')

{'Quotes': [{'QuoteId': 1,
   'MinPrice': 47,
   'Direct': True,
   'OutboundLeg': {'CarrierIds': [50441],
    'OriginId': 44759,
    'DestinationId': 66270,
    'DepartureDate': '2021-12-10T00:00:00'},
   'QuoteDateTime': '2021-02-23T15:42:00'}],
 'Carriers': [{'CarrierId': 50441, 'Name': 'easyJet'}],
 'Places': [{'Name': 'Paris Charles de Gaulle',
   'Type': 'Station',
   'PlaceId': 44759,
   'IataCode': 'CDG',
   'SkyscannerCode': 'CDG',
   'CityName': 'Paris',
   'CityId': 'PARI',
   'CountryName': 'France'},
  {'Name': 'London Luton',
   'Type': 'Station',
   'PlaceId': 66270,
   'IataCode': 'LTN',
   'SkyscannerCode': 'LTN',
   'CityName': 'London',
   'CityId': 'LOND',
   'CountryName': 'United Kingdom'},
  {'Name': 'Paris',
   'Type': 'City',
   'PlaceId': 3579769,
   'SkyscannerCode': 'PARI',
   'CityId': 'PARI',
   'CountryName': 'France'}],
 'Currencies': [{'Code': 'USD',
   'Symbol': '$',
   'ThousandsSeparator': ',',
   'DecimalSeparator': '.',
   'SymbolOnLeft': True,
   

In [6]:
from  time import sleep

def flight_days(origin, destination, start, end):
    dates = get_dates(start, end)
    my_dict = {}
    for date in list(dates):
        #result = None
        #while (result == None):
            try: 
                #print(date)
                #print("Gathering fligths")
                result = flight_prices(origin, destination, date)
                my_dict[date] = result
                sleep(5)
            except: 
                print("Error from the API")
                sleep(60)
    return my_dict

In [7]:
flights = flight_days('Barcelona','London','2021-11-26','2021-12-10')
flights

{'2021-11-26': {'Quotes': [{'QuoteId': 1,
    'MinPrice': 27,
    'Direct': True,
    'OutboundLeg': {'CarrierIds': [50441],
     'OriginId': 42414,
     'DestinationId': 66270,
     'DepartureDate': '2021-11-26T00:00:00'},
    'QuoteDateTime': '2021-02-23T15:41:00'}],
  'Carriers': [{'CarrierId': 50441, 'Name': 'easyJet'}],
  'Places': [{'Name': 'Barcelona',
    'Type': 'Station',
    'PlaceId': 42414,
    'IataCode': 'BCN',
    'SkyscannerCode': 'BCN',
    'CityName': 'Barcelona',
    'CityId': 'BARC',
    'CountryName': 'Spain'},
   {'Name': 'London Luton',
    'Type': 'Station',
    'PlaceId': 66270,
    'IataCode': 'LTN',
    'SkyscannerCode': 'LTN',
    'CityName': 'London',
    'CityId': 'LOND',
    'CountryName': 'United Kingdom'}],
  'Currencies': [{'Code': 'USD',
    'Symbol': '$',
    'ThousandsSeparator': ',',
    'DecimalSeparator': '.',
    'SymbolOnLeft': True,
    'SpaceBetweenAmountAndSymbol': False,
    'RoundingCoefficient': 0,
    'DecimalDigits': 2}]},
 '2021-11-27

In [19]:
my_dict = {}

# We iterate over the dates
for date in flights:
    print("The flights for {} are: ".format(date))
    #print('Price: ', flights[date]['Quotes'][0]['MinPrice'], '\nName origin: ', flights[date]['Places'][0]['Name'], '\nName arrival: ', flights[date]['Quotes'][0]['CarrierIds'], '\nAirline: ', 
    print('Price:', flights[date]['Quotes'][0]['MinPrice'],'\nDeparture Airport:', flights[date]['Places'][0]['Name'],'\nArrival Airport:', flights[date]['Places'][1]['Name'],'\nAirline:', flights[date]['Carriers'][0]['Name'],'\n')
    my_dict[date] = [] #We create an empty list of prices for the given date.
    min_price = 10000
    best_date = ""
    for elem in flights[date]['Quotes']: # We iterate over "Quotes" of each day.
        my_dict[date].append(elem['MinPrice']) # We fill the date with the prices.
        if (elem['MinPrice'] == min_price):
            min_price = elem['MinPrice']
            best_date = date
    #print("The prices are the same for this date: ",my_dict[date])
    try:
        print("The minimum price for this date is: ",min(my_dict[date]),'\n','\n----------------','\n') # We determine the minimum price.
    except:
        continue


The flights for 2021-11-26 are: 
Price: 27 
Departure Airport: Barcelona 
Arrival Airport: London Luton 
Airline: easyJet 

The minimum price for this date is:  27 
 
---------------- 

The flights for 2021-11-27 are: 
Price: 27 
Departure Airport: Barcelona 
Arrival Airport: London Luton 
Airline: easyJet 

The minimum price for this date is:  27 
 
---------------- 

The flights for 2021-11-28 are: 
Price: 27 
Departure Airport: Barcelona 
Arrival Airport: London Gatwick 
Airline: easyJet 

The minimum price for this date is:  27 
 
---------------- 

The flights for 2021-11-29 are: 
Price: 27 
Departure Airport: Barcelona 
Arrival Airport: London Gatwick 
Airline: easyJet 

The minimum price for this date is:  27 
 
---------------- 

The flights for 2021-11-30 are: 
Price: 27 
Departure Airport: Barcelona 
Arrival Airport: London Gatwick 
Airline: easyJet 

The minimum price for this date is:  27 
 
---------------- 

The flights for 2021-12-01 are: 
Price: 27 
Departure Airport: B

In [28]:
def repeat(x):
    _size = len(x)
    repeated = []
    for i in range(_size):
        k = i + 1
        for j in range(k, _size):
            if x[i] == x[j] and x[i] not in repeated:
                repeated.append(x[i])
    return repeated

print(repeat(store_price))

[27, 24]


In [26]:
import re
my_dict = {}
store_dict = []
store_price = []

for date in flights: # We iterate over the dates
    my_dict[date] = [] #We create an empty list of prices for the given date.
    min_price = 10000
    best_date = ""
    
    for elem in flights[date]['Quotes']: # We iterate over “Quotes” of each day.
        my_dict[date].append(elem['MinPrice']) # We fill the date with the prices.
        if (elem['MinPrice'] == min_price):
            min_price = elem['MinPrice']
            best_date = date
    try:
        store_dict.append([{'Price': flights[date]['Quotes'][0]['MinPrice'],
                           'Name origin': flights[date]['Places'][0]['Name'],
                           'Name arrival': flights[date]['Places'][1]['Name'],
                           'Airline': flights[date]['Carriers'][0]['Name'],
                           'Date': flights[date]['Quotes'][0]['OutboundLeg']['DepartureDate'][:-9]
                           }])
        #print(“The min price for this date is: “,min(my_dict[date])) # We determine the minimum price.
    except:
        continue
        
for elem in store_dict:
    store_price.append(elem[0]['Price'])
    
for i in store_dict:
    if i[0]['Price'] in repeat(store_price):
        print(i)

[{'Price': 27, 'Name origin': 'Barcelona', 'Name arrival': 'London Luton', 'Airline': 'easyJet', 'Date': '2021-11-26'}]
[{'Price': 27, 'Name origin': 'Barcelona', 'Name arrival': 'London Luton', 'Airline': 'easyJet', 'Date': '2021-11-27'}]
[{'Price': 27, 'Name origin': 'Barcelona', 'Name arrival': 'London Gatwick', 'Airline': 'easyJet', 'Date': '2021-11-28'}]
[{'Price': 27, 'Name origin': 'Barcelona', 'Name arrival': 'London Gatwick', 'Airline': 'easyJet', 'Date': '2021-11-29'}]
[{'Price': 27, 'Name origin': 'Barcelona', 'Name arrival': 'London Gatwick', 'Airline': 'easyJet', 'Date': '2021-11-30'}]
[{'Price': 27, 'Name origin': 'Barcelona', 'Name arrival': 'London Gatwick', 'Airline': 'easyJet', 'Date': '2021-12-01'}]
[{'Price': 24, 'Name origin': 'Barcelona', 'Name arrival': 'London Luton', 'Airline': 'easyJet', 'Date': '2021-12-02'}]
[{'Price': 24, 'Name origin': 'Barcelona', 'Name arrival': 'London Luton', 'Airline': 'easyJet', 'Date': '2021-12-03'}]
[{'Price': 27, 'Name origin': 'B